## Упражнение 1
Проверьте, возможно ли изменить список в процессе итерирования.

In [1]:
l = [1, 2, 3, 4, 5, 6, 7]
for i in l:
    if i%2 == 0:
        l.append(i*2 + 1)
    print(i, end=' ')

1 2 3 4 5 6 7 5 9 13 

Как видно из кода выше, список можно изменять во время итерирования

## Упражнение 2
Реализуйте класс BinTree двоичного дерева, итерирование по которму происходит в порядке обхода в глубину.


In [2]:
class Node:
    def __init__(self, value, l=None, r=None):
        self.value = value
        self.l = l
        self.r = r
    
    def insert(self, data):
        if self.value > data:
            if self.l is None:
                self.l = Node(data)
            else:
                self.l.insert(data)
        else:
            if self.r is None:
                self.r = Node(data)
            else:
                self.r.insert(data)
                
    def show(self):
        if self.l is not None:
            self.l.show()
        print(self.value)
        if self.r is not None:
            self.r.show()        
        
    def get_value(self):
        return self.value

    def get_l(self):
        return self.l
    
    def get_r(self):
        return self.r

class BinTree:
    def __init__(self):
        self.root = None
        self.length = 0
        self.queue = []

    def add(self, data):
        if self.root is None:
            self.root = Node(data)
        else:
            self.root.insert(data)
    
    def show(self):
        if self.root is not None:
            self.root.show()
        
    def __iter__(self):
        if self.root is None:
            return self
        self.queue = [self.root]
        while self.queue[0].l is not None:
            self.queue.insert(0, self.queue[0].l)
        return self

    def __next__(self):
        if not self.queue:
            raise StopIteration()
        val = self.queue[0].get_value()
        if self.queue[0].r is not None:
            self.queue[0] = self.queue[0].r
            while self.queue[0].l is not None:
                self.queue.insert(0, self.queue[0].l)
        else:
            self.queue.pop(0)
        return val

In [3]:
bt = BinTree()
bt.add(5)
bt.add(1)
bt.add(4)
bt.add(9)
bt.add(10)
bt.add(1)
bt.add(2)
bt.add(6)
bt.add(3)
bt.add(8)
for i in bt:
    print(i)

1
1
2
3
4
5
6
8
9
10


## Упражнение 3
Одним из важнейших применений генераторов является загрузка наборов данных "на лету", без необходимости загрузки в память всего набора. Попробуйте решить похожую задачу.

Вам необходимо создать класс ```TextLoader```, который принимает в инициализаторе адрес папки. Метод ```__len__``` должен возвращать количество текстов в папке. метод ```__getitem__``` загружает текст, приводит его к нижнему регистру и убирает знаки препинания, при итерировании возвращаются нормализованные тексты, аналогично методу ```__getitem__```.

In [4]:
import os, os.path
import string

class TextLoader:
    def __init__(self, path):
        self.path = path
    
    def __len__(self):
        return len(os.listdir(self.path))
    
    def __readAndNormolize(filePath):
            with open(filePath, 'r') as f:
                return f.read().lower().translate(str.maketrans('', '', string.punctuation))
            
    def __getitem__(self, idx):
        return TextLoader.__readAndNormolize(self.path + os.listdir(self.path)[idx])
        
    def __iter__(self):
        self.currentFile = iter(os.listdir(self.path))
        return self
    
    def __next__(self):
        return TextLoader.__readAndNormolize(self.path + next(self.currentFile))

In [5]:
tl = TextLoader('./sample/')
print(len(tl))
print(tl[2])
print()
for i in tl:
    print(i)

100
подходит маленький сын к отцу и спрашивает  пап скажи что такое политика  объясню тебе на примере нашей семьи вот смотри я зарабатываю деньги значит я министерство финансов мама ведет хозяйство значит она правительство дедушка следит чтобы в доме был порядок  он профсоюзы домработница выполняет всю работу по дому  она рабочий класс

49	блондинки беседуют
— а как ты думаешь зачем на вертолете ставят такой большой пропеллер
— ты что это же вентилятор он нужен для того чтоб пилот не потел
— чушь ты меня разыгрываешь
— нет не чушь я совсем недавно летела на вертолете так вот перед самым приземлением этот пропеллер вдруг взял и перестал крутиться ты бы видела как пилот вспотел сразу

шумная улица по ней идет крутейная блондинка из глубокого декольте торчит прекрасной формы грудь народ косится шарахается по сторонам а блондинке — хоть бы что — идет руками машет ну один наш героический милиционер решился — подходит представляется — все по форме потом говорит «гражданочка чтойто у вас внеш

## Упражнение 4
От некоторого устройства в режиме реального времени приходят данные. Необходимо написать сопрограмму, которая вычисляет среднее, дисперсию, а также количество элементов в переданном наборе данных с устройства. Результаты работы сопрограмма должна выдавать при отправке соответствующих сигналов.

In [6]:
import numpy

class PrintАverage(Exception):
    pass

class PrintVariance(Exception):
    pass

class PrintCount(Exception):
    pass

def device_control_coroutine():
    collection = []
    try:
        while True:
            try:
                x = yield
                if x is not None:
                    collection.append(x)
            except PrintАverage:
                yield numpy.average(collection)
            except PrintVariance:
                yield numpy.var(collection)
            except PrintCount:
                yield len(collection)
    finally:
        print("Stop coroutine")

In [7]:
coroutine = device_control_coroutine()
next(coroutine)
for i in range(25):
    coroutine.send(i)
    if i%2 == 0:
        print("Average:", coroutine.throw(PrintАverage))
        next(coroutine)
    if i%3 == 0:
        print("Variance:", coroutine.throw(PrintVariance))
        next(coroutine)
    if i%5 == 0:
        print("Count:", coroutine.throw(PrintCount))
        next(coroutine)

print()
print("Average:", coroutine.throw(PrintАverage))
next(coroutine)
print("Variance:", coroutine.throw(PrintVariance))
next(coroutine)
print("Count:", coroutine.throw(PrintCount))

coroutine.close()

Average: 0.0
Variance: 0.0
Count: 1
Average: 1.0
Variance: 1.25
Average: 2.0
Count: 6
Average: 3.0
Variance: 4.0
Average: 4.0
Variance: 8.25
Average: 5.0
Count: 11
Average: 6.0
Variance: 14.0
Average: 7.0
Variance: 21.25
Count: 16
Average: 8.0
Average: 9.0
Variance: 30.0
Average: 10.0
Count: 21
Variance: 40.25
Average: 11.0
Average: 12.0
Variance: 52.0

Average: 12.0
Variance: 52.0
Count: 25
Stop coroutine


## Упражнение 5
Представьте, что у вас настроено взаимодействие с сервером, от которого приходят пакеты, содержащие сообщения от различных клиентов. Обработка каждого из клиентов должна идти в отдельном потоке.

Реализуйте:

1. Корутина ```connect_user``` принимает данные авторизации от пользователя, открывает файл с названием .txt и создает на его основе корутину ```write_to_file```
2. Корутина ```write_to_file(f_obj)``` записывает переданное планировщиком задач сообщение пользователя, которые записываются в файловый объект, переданный в качестве аргумента при генерации. Также принимает и обрабатывает сигнал об окончании соединения и выходит из сопрограммы.
4. Планировщик задач, распределяющий задачи по сопроцессам на каждого пользователя.

In [8]:
def user_connection(username):
    import random
    for i in range(random.randint(10, 20)):
        yield f"{username} message{i}"

def establish_connection(auth=True):
    import random
    id = f"{random.randint(0,100000000):010}"
    if auth:
        yield f"auth {id}"
    yield from user_connection(id)
    if auth:
        yield f"disconnect {id}"
        
def connection():
    import random
    connections = [establish_connection(True) for i in range(10)]
    connections.append(establish_connection(False))
    connections.append(establish_connection(False))
    while len(connections):
        conn = random.choice(connections)
        try:
            yield next(conn)
        except StopIteration:
            del connections[connections.index(conn)]

In [9]:
def write_to_file(f_obj):
    while True:
        m = yield
        f_obj.write(m+'\n')

def connect_user():
    while True:
        id = yield
        f = open('./messages/'+id+'.txt', 'w')
        w_c = write_to_file(f)
        next(w_c)
        yield w_c
    
def task_coroutine():
    authorized = {}
    user_corotine = connect_user()
    next(user_corotine)
    while True:
        a, b = (yield).split()
        if a == 'auth':
            authorized[b] = user_corotine.send(b)
            next(user_corotine)
        elif a == 'disconnect':
            authorized.pop(b, None)
        else:
            try:
                authorized[a].send(b)
            except KeyError:
                None

In [10]:
c = task_coroutine()
next(c)
for i in connection():
    c.send(i)
    
tl = TextLoader('./messages/')
for t in tl:
    print(t)

message0
message1
message2
message3
message4
message5
message6
message7
message8
message9
message10
message11
message12
message13
message14
message15
message16
message17
message18

message0
message1
message2
message3
message4
message5
message6
message7
message8
message9
message10
message11
message12
message13
message14
message15
message16
message17
message18
message19

message0
message1
message2
message3
message4
message5
message6
message7
message8
message9
message10
message11
message12
message13
message14
message15
message16
message17

message0
message1
message2
message3
message4
message5
message6
message7
message8
message9
message10
message11
message12
message13
message14
message15
message16


message0
message1
message2
message3
message4
message5
message6
message7
message8
message9
message10
message11
message12
message13
message14
message15
message16
message17
message18
message19

message0
message1
message2
message3
message4
message5
message6
message7
message8
message9
message10
mess